# Advisors

Advisors are an interesting feature in Spring-AI that allows you to flexibly intercept,
modify, and enhance AI interactions.

With `Advisors`, you can:
- Add necessary context to user requests
- Filter out harmful or sensitive content in AI requests
- Track custom metrics
- Ensure consistent output structure
- And more

Let's add dependencies and create a `ChatModel`

In [1]:
%useLatestDescriptors
%use spring-ai-openai

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_5_CHAT_LATEST)
    .temperature(0.7)
    .build()


val chatModel = OpenAiChatModel.builder()
    .openAiApi(openAiApi)
    .defaultOptions(openAiOptions)
    .build()

Let's add the `MessageChatMemory` advisor.
As the name suggests, this advisor will implement message history,
preserving the conversation context.
For this Advisor, we'll need a `ChatMemory` instance
where messages will be stored.

In [3]:
val chatMemory = MessageWindowChatMemory.builder().build()

val chatClient = ChatClient
    .builder(chatModel)
    .defaultAdvisors(MessageChatMemoryAdvisor.builder(chatMemory).build())
    .build()


Let's test how this works

In [4]:
chatClient.prompt("Hi, tell me a joke").call().content()

Sure! Here’s one for you:  

Why don’t skeletons fight each other?  

Because they don’t have the guts. 🦴😄

In [5]:
chatClient.prompt("What is previous message in our chat history?").call().content()

The previous message in our chat history is the joke I just told you:  

"Why don’t skeletons fight each other? Because they don’t have the guts."

As we can see, the LLM now has access to our message history.

Spring-AI includes several predefined Advisors:
- `MessageChatMemoryAdvisor`
- `PromptChatMemoryAdvisor`
- `QuestionAnswerAdvisor`
- `RetrievalAugmentationAdvisor`
- `SafeGuardAdvisor`
- `SimpleLoggerAdvisor`
- `VectorStoreChatMemoryAdvisor`

And you can create your own custom Advisor as well.

Let's do that now.
We'll create an Advisor that logs requests and responses by outputting them to our console.
To do this, we'll extend `CallAroundAdvisor` and implement the `aroundCall` method

In [6]:
import org.springframework.ai.chat.client.advisor.api.CallAdvisor
import org.springframework.ai.chat.client.advisor.api.CallAdvisorChain

class CustomLogger: CallAdvisor {
    override fun getName(): String {
        return "CustomLogger"
    }

    override fun getOrder(): Int = 0

    override fun adviseCall(chatClientRequest: ChatClientRequest, callAdvisorChain: CallAdvisorChain): ChatClientResponse {
        println("CustomLogger.Before: ${chatClientRequest}")
        val chatClientResponse = callAdvisorChain.nextCall(chatClientRequest)
        println("CustomLogger.After: ${chatClientResponse}")
        return chatClientResponse
    }
}

Now let's see our `CustomAdvisor` in action

In [7]:
chatClient
    .prompt("Generate HelloWorld in Kotlin")
    .advisors(CustomLogger())
    .call()
    .content()

CustomLogger.Before: ChatClientRequest[prompt=Prompt{messages=[UserMessage{content='Hi, tell me a joke', metadata={messageType=USER}, messageType=USER}, AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=Sure! Here’s one for you:  

Why don’t skeletons fight each other?  

Because they don’t have the guts. 🦴😄, metadata={role=ASSISTANT, messageType=ASSISTANT, finishReason=STOP, refusal=, index=0, annotations=[], id=chatcmpl-DB6ywKthy6COgW1i2xI9b7IqXqr8L}], UserMessage{content='What is previous message in our chat history?', metadata={messageType=USER}, messageType=USER}, AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=The previous message in our chat history is the joke I just told you:  

"Why don’t skeletons fight each other? Because they don’t have the guts.", metadata={role=ASSISTANT, messageType=ASSISTANT, finishReason=STOP, refusal=, index=0, annotations=[], id=chatcmpl-DB6yxncMeSQ4K72dn6eA5qMLFc1lD}], UserMessage{content='Generate HelloWorld in 

Sure! Here’s a simple **Hello World** program written in Kotlin:

```kotlin
fun main() {
    println("Hello, World!")
}
```

When you run this program, it will print:

```
Hello, World!
```